# **Aggregate Dataset for Machine Learning**

## Import Library

In [3]:
import pandas as pd
import numpy as np

## Load Dataset

In [4]:
df_korban = pd.read_csv('Data Korban Banjir (databoks, dll)/Data_Korban_Banjir_Meninggal.csv')
df_hutan_gfw = pd.read_csv('Data Deforestasi Aceh - HAKA (Pemerintah Aceh)/Aceh 2 tree cover loss (GFW) - Subnational 2 tree cover loss.csv')
df_lahan = pd.read_csv('Kemiringan Lahan Aceh (Perkim)/Data Kemiringan lahan Aceh.csv')
df_penduduk = pd.read_csv('Data Kepadatan Penduduk Aceh (BPS Statistik)/Kepadatan_Penduduk.csv')

## Cleaning Function

In [5]:
def clean_kabupaten(nama):
    if pd.isna(nama): return ""
    nama = str(nama).lower()
    nama = nama.replace("kab.", "").replace("kabupaten", "").replace("kota", "")
    nama = nama.strip()
    return nama

df_hutan_gfw['kab_clean'] = df_hutan_gfw['subnational2'].apply(clean_kabupaten)
df_lahan['kab_clean'] = df_lahan['kabupaten'].apply(clean_kabupaten)
df_penduduk['kab_clean'] = df_penduduk['kabupaten'].apply(clean_kabupaten)
df_korban['kab_clean'] = df_korban['kabupaten_kota'].apply(clean_kabupaten)

## Big Data Preparation

### Ambil data kepadatan penduduk 2024 (tahun terakhir yg didapat)

In [6]:
df_master = df_penduduk[df_penduduk['tahun'] == 2024][['kabupaten', 'kab_clean', 'kepadatan_penduduk_km2']].copy()

if df_master.empty:
    print("⚠️ Info: Data 2024 tidak terbaca otomatis, menggunakan data tahun terbaru yang tersedia.")
    th_terbaru = df_penduduk['tahun'].max()
    df_master = df_penduduk[df_penduduk['tahun'] == th_terbaru][['kabupaten', 'kab_clean', 'kepadatan_penduduk_km2']].copy()

df_master.drop_duplicates(subset='kab_clean', inplace=True)

### Data Korban

In [7]:
data_korban = df_korban.groupby('kab_clean')['nilai'].sum().reset_index()
data_korban.rename(columns={'nilai': 'total_korban'}, inplace=True)

### Data Deforestasi

In [8]:
cols_deforest = [c for c in df_hutan_gfw.columns if 'tc_loss_ha_' in c and int(c.split('_')[-1]) >= 2019]
df_hutan_aceh = df_hutan_gfw[(df_hutan_gfw['subnational1'] == 'Aceh') & (df_hutan_gfw['threshold'] == 30)].copy()
df_hutan_aceh['total_deforestasi'] = df_hutan_aceh[cols_deforest].sum(axis=1)
data_hutan = df_hutan_aceh.groupby('kab_clean')['total_deforestasi'].sum().reset_index()

### Data Kemiringan Lahan

In [9]:
data_lahan = df_lahan.groupby('kab_clean')['kemiringan_rata2_persen'].mean().reset_index()

In [16]:
df_final = (
    df_master
    .merge(data_korban, on='kab_clean', how='left')
    .merge(data_hutan, on='kab_clean', how='left')
    .merge(data_lahan, on='kab_clean', how='left')
)

# cek cepat
df_final.head()

,kabupaten,kab_clean,kepadatan_penduduk_km2,total_korban,total_deforestasi,kemiringan_rata2_persen
0,Simeulue,simeulue,48,NaN,3583,22.500000
1,Aceh Singkil,aceh singkil,62,NaN,9205,3.000000
2,Aceh Selatan,aceh selatan,63,NaN,13904,22.500000
3,Aceh Tenggara,aceh tenggara,55,13.0,4783,21.333333
4,Aceh Timur,aceh timur,71,53.0,21155,8.000000


In [17]:
df_tampilan = df_final.copy()

## Format Kolom

### Format Kolom Korban

In [18]:
df_tampilan['total_korban'] = df_tampilan['total_korban'].apply(
lambda x: "Data Tidak Tersedia" if pd.isna(x) else int(x)
)

### Format Deforestasi

In [19]:
df_tampilan['total_deforestasi'] = df_tampilan['total_deforestasi'].apply(
    lambda x: "-" if pd.isna(x) else "{:,.0f} Ha".format(x)
)

### Format Kemiringan

In [20]:
df_tampilan['kemiringan_rata2_persen'] = df_tampilan['kemiringan_rata2_persen'].apply(
    lambda x: "-" if pd.isna(x) else "{:.1f}%".format(x)
)

### Format Kepadatan Penduduk

In [21]:
df_tampilan['kepadatan_penduduk_km2'] = df_tampilan['kepadatan_penduduk_km2'].astype(str)

In [22]:
df_tampilan = df_tampilan[['kabupaten', 'total_korban', 'total_deforestasi', 'kemiringan_rata2_persen', 'kepadatan_penduduk_km2']]
display(df_tampilan)

,kabupaten,total_korban,total_deforestasi,kemiringan_rata2_persen,kepadatan_penduduk_km2
0,Simeulue,Data Tidak Tersedia,"3,583 Ha",22.5%,48
1,Aceh Singkil,Data Tidak Tersedia,"9,205 Ha",3.0%,62
2,Aceh Selatan,Data Tidak Tersedia,"13,904 Ha",22.5%,63
3,Aceh Tenggara,13,"4,783 Ha",21.3%,55
4,Aceh Timur,53,"21,155 Ha",8.0%,71
5,Aceh Tengah,24,"12,206 Ha",30.0%,53
6,Aceh Barat,Data Tidak Tersedia,"15,788 Ha",3.0%,71
7,Aceh Besar,Data Tidak Tersedia,"6,652 Ha",25.0%,144
8,Pidie,Data Tidak Tersedia,"5,218 Ha",8.0%,147
9,Bireuen,33,"8,018 Ha",8.0%,241


##### <span style="color:red;"> Angka **0** dalam data tersebut belum tentu total korban benar-benar 0. 0 dalam tabel tersebut bisa berarti data tidak tersedia, data tidak akurat, atau mungkin memang tidak terdeteksi korban di daerah tersebut</span>


## Convert to CSV

In [ ]:
df_ml = df_final.copy()
df_ml['total_korban'] = df_ml['total_korban'].fillna(0) 
df_ml['total_deforestasi'] = df_ml['total_deforestasi'].fillna(df_ml['total_deforestasi'].mean())
df_ml['kemiringan_rata2_persen'] = df_ml['kemiringan_rata2_persen'].fillna(df_ml['kemiringan_rata2_persen'].mean())

df_ml.to_csv('Data_Siap_Clustering.csv', index=False)